In [1]:
# import scipy
import scipy.optimize

from math import pi
from math import sqrt
from math import sin

import numpy as np
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

#import Ipynb_importer
#import Chiara as H55

from pyDOE import *
import numpy as np

def surrogate(beta_r,c,j_twist, v,nD,k=0.5) :
    """ Parameter k was applied in the process of generating the surrogate in order to have every aprameter belonging to [0,1]. 
    So to generate correct results we need to apply it once again in our polynomial expression 
    beta_r : global pitch (rad)
    c : chord root (m)
    j_twist : articifal advance parameter for blade twist (-)
    v : aircraft speed (m/s)
    nD : rotationnal ratio (m/s)
    """
    # Cp stands for the power coefficient of a propeller (-)
    cp = 0.01391 + 5.38589*c*(beta_r**2) - 6.45660*c*(k*(v/nD))**2 + 0.00023*c*beta_r*(k*j_twist) + 3.96380*c*(k*v/(nD))*beta_r + 0.27890*c*(k*j_twist) + 0.17054*(beta_r**3) -0.15440*c \
    + 0.70854*(c**2) + 0.55346*c*(k*v/(nD))*(k*j_twist) - 0.05988*beta_r - 5.92912*(c**2)*beta_r + 1.52780*c*beta_r + 0.09312*(j_twist*k)*(beta_r**2) - 0.26690*c*(k*j_twist)**2 
    
    # Ct stands for the thrust coefficient of a propeller (-)
    ct = 0.02710 + 2.35828*c*(beta_r**2) - 3.21663*c*(k*v/(nD))**2 + 6.03656*c*beta_r + 0.10240*(k*j_twist)*(beta_r)**2 - 0.15488*(k*v/(nD)) - 14.28915*(c**2)*beta_r - 3.31653*c*(k*v/(nD)) \
    - 1.74088*(c**2)*(k*j_twist)+0.27254*(k*v/(nD))*beta_r+0.10243*(k*j_twist)*beta_r - 0.19851*beta_r*(k*j_twist)**2 + 0.65639*c*(k*j_twist) + 9.25027*(c**2)*(k*v/(nD))-0.15915*(beta_r**3)
    return ct,cp

pd.options.display.float_format = '{:,.2f}'.format

In [4]:
def sizing_code():
# Design variables
# ---
    J_SL = 0.5              #current density [0, 1] 0 the motor is off, 1 full power
    d_over_l_SL = 0.5       #form factor [0.1,1]*2 <=> [0.2,2]
    t_flight =30/60         #t_flight in hours

#Propeller parameters
# ---
    beta_r = 17.92 * np.pi / 180       #optimum beta for the choosen RPM and SHP find with BEMT Code
    c_D = 0.07
    j_twist = 0.93
    v = 55                             #CLIMB SPEED
    RPM = 2200
    n = RPM/60
    R = 1                              #PROPELLER RADIUS
    D = 2*R
    k = 0.5
    rho = 1.225
    omega = n*2*np.pi
    J = v/(n*D)
    ct,cp = surrogate(beta_r,c_D,j_twist,v,n*D, k=0.5)    #experimental model to obtain ct and cp 
    Power_prop = rho*(n**3)*(D**5)*cp # Power required by propeller
    Thrust = rho*(n**2)*(D**4)*ct
    Torque = Power_prop/(2*np.pi*n)
    
    Power = Power_prop #*eta_gear (almost = 1)  

    #Reference motor SIEMENS SP-260D
    Torque_ref = 1000                                                                       # (Nm)
    RPM_ref = 2500                                                                          # (Tr/min)
    Pj_ref = 10000                                                                          # Joule losses (W)
    Pi_ref = 4000                                                                           # Iron losses (W)
    Pc_ref = 14000                                                                          # Total losses (W)
    m_motor_ref = 50                                                                        # Motor mass (kg)
    d_ref = 0.418                                                                           # Diameter (m)
    l_ref = 0.300                                                                           # Length (m)
    Power_ref = 261000                                                                      # Motor power (W)
    f_ref = 750
    
    #Motor Scaling law (_SL)
    # ---
    eff_SL = 1
    eff_SL_last = 0
    while(abs(eff_SL-eff_SL_last) > 0.001): #loop that converge to the efficiency value of the propeller - WHEN IT CONVERGES
        omega_SL = RPM/RPM_ref 
        Power_corr = Power*(2-eff_SL) #real amount of power needed taking into accont the motor efficiency
        Torque_SL = (Power_corr/Power_ref)/omega_SL

        #Hypothesis
        B_SL = 1
        m_iron_SL = 1
        d_SL = ((Torque_SL*d_over_l_SL)/(J_SL*B_SL))**(1/3)
        l_SL = d_SL/d_over_l_SL

        #parameters
        m_motor_SL = d_SL*l_SL
        f_SL = omega_SL*d_SL
        Pi_SL = (f_SL**1.5)*m_iron_SL   #FROM IMECHE
        Pj_SL = (J_SL**2)*d_SL*l_SL     #FROM IMECHE
        Pc = Pj_SL*Pj_ref + Pi_SL*Pi_ref
        Pc_SL = Pc/Pc_ref
        kc_SL = Pc_SL/(d_SL*l_SL)
        
        #keep last efficiency value and finding the new one for the loop
        eff_SL_last = eff_SL
        eff_SL = 1/(1+(12/49)*(d_SL**(5/6))*(l_SL**(1/3))*(kc_SL**(-1/6)))
    
    
    c_bat = 500 #Wh/kg  LOOK AT FOLDER MALLA 'APRELIMINARYSIZINGMETHOD...' pag 20
    m_bat = (Power_corr*t_flight)/c_bat


    #PRINT THE RESULTS IN THE TABLE
    col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']
    df = pd.DataFrame()
    df = df.append([{'Type': 'Variables', 'Name': 'V', 'Value': v, 'Unit': '[m/s]', 'Comment': 'Airspeed'}])[col_names]
    df = df.append([{'Type': 'Propeller', 'Name': 'Beta', 'Value': beta_r, 'Unit': '[rad]', 'Comment': 'propeller pitch'}])[col_names]
    df = df.append([{'Type': 'Propeller', 'Name': 'chord to diameter', 'Value': c_D, 'Unit': '[-]', 'Comment': 'root chord over diameter'}])[col_names]
    df = df.append([{'Type': 'Propeller', 'Name': 'J twist', 'Value': j_twist, 'Unit': '[-]', 'Comment': 'Advance parameter for propeller twist'}])[col_names]
    df = df.append([{'Type': 'Propeller', 'Name': 'J', 'Value': J, 'Unit': '[-]', 'Comment': 'Advance parameter'}])[col_names]
    df = df.append([{'Type': 'Propeller', 'Name': 'R', 'Value': R, 'Unit': '[m]', 'Comment': 'Radius'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Propeller thrust', 'Value': Thrust, 'Unit': '[N]', 'Comment': 'Thrust'}])[col_names]
    df = df.append([{'Type': 'Variables', 'Name': 'rho', 'Value': rho, 'Unit': '[kg/m^3]', 'Comment': 'Air density'}])[col_names]
    df = df.append([{'Type': 'Propeller', 'Name': 'power', 'Value': Power, 'Unit': '[W]', 'Comment': 'power'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Propeller torque', 'Value': Torque, 'Unit': '[Nm]', 'Comment': 'Torque'}])[col_names]
    df = df.append([{'Type': 'Propeller', 'Name': 'ct', 'Value': ct, 'Unit': '[-]', 'Comment': 'Thrust coefficient of the propeller'}])[col_names]
    df = df.append([{'Type': 'Propeller', 'Name': 'cp', 'Value': cp, 'Unit': '[-]', 'Comment': 'Power coefficient of the propeller'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Magnetic field', 'Value': B_SL, 'Unit': '[Tesla]', 'Comment': 'Magnetic field / Magnetic field ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Current density', 'Value': J_SL, 'Unit': '[A/m^2]', 'Comment': 'j / j_ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'D/L', 'Value': d_over_l_SL, 'Unit': '[-]', 'Comment': 'Form factor'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Omega', 'Value': omega_SL*(RPM_ref/60*D), 'Unit': '[m/s]', 'Comment': 'omega / omega ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Motor Diameter', 'Value': d_SL*d_ref, 'Unit': '[m]', 'Comment': 'Diameter / Diameter ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Motor Length', 'Value': l_SL*l_ref, 'Unit': '[m]', 'Comment': 'Length / Length ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Frequency', 'Value': f_SL*f_ref, 'Unit': '[Hz]', 'Comment': 'Frequency / Frequency ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Joule losses', 'Value': Pj_SL*Pj_ref, 'Unit': '[W]', 'Comment': 'Pj / Pj ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Iron losses', 'Value': Pi_SL*Pi_ref, 'Unit': '[W]', 'Comment': 'Pi / P ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Cooling power', 'Value': Pc_SL*Pc_ref, 'Unit': '[W]', 'Comment': 'Pc / Pc ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Cooling system efficiency', 'Value': kc_SL, 'Unit': '[-]', 'Comment': 'kc / kc ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Motor power', 'Value': Power_corr, 'Unit': '[W]', 'Comment': 'Power corrected by the motor efficiency'}])[col_names]
    #df = df.append([{'Type': 'Motor', 'Name': 'Motor torque', 'Value': Torque_SL*Torque_ref, 'Unit': '[Nm]', 'Comment': 'T / T ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Efficiency', 'Value': eff_SL, 'Unit': '[-]', 'Comment': 'Efficiency of the motor'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Motor mass', 'Value': m_motor_SL*m_motor_ref, 'Unit': '[kg]', 'Comment': 'm_motor / m_motor ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Battery mass', 'Value': m_bat, 'Unit': '[kg]', 'Comment': 'Battery mass onboard'}])[col_names]

    items = sorted(df['Type'].unique().tolist())
    def f(Type):
         return df[df['Type']==Type] 
    widgets.interact(f, Type=items)
    return f

In [5]:
sizing_code()

C:\Users\Chiara\AppData\Local\Temp\ipykernel_27424\1363974663.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Variables', 'Name': 'V', 'Value': v, 'Unit': '[m/s]', 'Comment': 'Airspeed'}])[col_names]
C:\Users\Chiara\AppData\Local\Temp\ipykernel_27424\1363974663.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Propeller', 'Name': 'Beta', 'Value': beta_r, 'Unit': '[rad]', 'Comment': 'propeller pitch'}])[col_names]
C:\Users\Chiara\AppData\Local\Temp\ipykernel_27424\1363974663.py:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Propeller', 'Name': 'chord to diameter', 'Value': c_D, 'Unit': '[-]', 'Comment': 'root chord over diameter'}])[col_

interactive(children=(Dropdown(description='Type', options=('Motor', 'Propeller', 'Variables'), value='Motor')…

<function __main__.sizing_code.<locals>.f(Type)>